In [ ]:
import os
import time
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome()

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

## 예매자 통계

In [ ]:
def initialize_driver():
    driver = webdriver.Chrome()
    return driver

def get_theater_links(driver, query):
    driver.get(query)
    time.sleep(1)
    number = 150

    addresslist = []

    for i in range(1, int(number) + 1):
        xpath = f"/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody/tr[{i}]/td[1]/a"
        elem = driver.find_element(By.XPATH, xpath).get_attribute("href")
        addresslist.append(elem)

    return addresslist

def scrape_reviews(driver, addresslist):
    scraped_data = []

    for lis in addresslist:
        driver.get(lis)
        time.sleep(1)

        title = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[1]/div/div[1]/h2").text
        male_percentage = None
        female_percentage = None
        age_percentages = {}

        try:
            gender_statistics = driver.find_elements(By.CLASS_NAME, "statGenderType")
            for stat in gender_statistics:
                gender_name = stat.find_element(By.CLASS_NAME, "statGenderName").text.strip()
                gender_percent = stat.find_element(By.CLASS_NAME, "statGenderValue").text.strip().replace("%", "")

                if gender_name == "남자":
                    male_percentage = gender_percent
                elif gender_name == "여자":
                    female_percentage = gender_percent
        except NoSuchElementException:
            pass
        try:
            age_section = driver.find_element(By.CLASS_NAME, "statAge")
            age_types = age_section.find_elements(By.CLASS_NAME, "statAgeType")
            for age_type in age_types:
                age_name = age_type.find_element(By.CLASS_NAME, "statAgeName").text.strip()
                age_percent = age_type.find_element(By.CLASS_NAME, "statAgePercent").text.strip().replace("%", "")
                age_percentages[age_name] = age_percent
        except NoSuchElementException:
            pass

        scraped_data.append({
            "title": title,
            "male_percentage": male_percentage,
            "female_percentage": female_percentage,
            "age_percentages": age_percentages,
        })

    return scraped_data

driver = webdriver.Chrome()

addresslist = get_theater_links(driver, "http://ticket.interpark.com/TPGoodsList.asp?Ca=Dra&Sort=3")

scraped_data = scrape_reviews(driver, addresslist)
df = pd.DataFrame(scraped_data)

driver.quit()

df.head(10)

,title,male_percentage,female_percentage,age_percentages
0,연극 너의 목소리가 들려,32.4,67.6,"{'10대': '3.4', '20대': '35.3', '30대': '35.3', '..."
1,대학로 1위 연극 〈쉬어매드니스〉,26.1,73.9,"{'10대': '11.9', '20대': '55.6', '30대': '20.8', ..."
2,행오버,32.2,67.8,"{'10대': '7.2', '20대': '56.7', '30대': '25.5', '..."
3,연극 〈바닷마을 다이어리〉,34.7,65.3,"{'10대': '3', '20대': '19.2', '30대': '31', '40대'..."
4,연극 한뼘사이,44.1,55.9,"{'10대': '8.1', '20대': '57', '30대': '24.4', '40..."
5,연극〈늘근도둑이야기〉,30.3,69.7,"{'10대': '3', '20대': '28.2', '30대': '25.6', '40..."
6,뮤직드라마 〈불편한 편의점〉,23.2,76.8,"{'10대': '2.4', '20대': '19.8', '30대': '20.6', '..."
7,코믹감동 휴먼판타지 연극 〈2호선 세입자〉,30.5,69.5,"{'10대': '6.7', '20대': '41.7', '30대': '22.3', '..."
8,10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀,40.3,59.7,"{'10대': '11.9', '20대': '52.4', '30대': '17.5', ..."
9,좀비연극 오마이갓,29.2,70.8,"{'10대': '8.7', '20대': '41.2', '30대': '31.9', '..."


In [ ]:
df.to_excel("C:/Users/82109/Documents/연극_통계.xlsx", sheet_name='Sheet1')

## 리뷰 (아이디, 평점, 날짜)

In [ ]:
# 모든 페이지—모든 리뷰 크롤링 (베스트 리뷰가 모든 페이지에 중복되서 나옴) + 연극 당 최대 크롤링 개수 지정

In [ ]:
def initialize_driver():
    driver = webdriver.Chrome()
    return driver

def get_theater_links(driver, query):
    driver.get(query)
    time.sleep(1)
    number = 150

    addresslist = []

    for i in range(1, int(number) + 1):
        xpath = f"/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody/tr[{i}]/td[1]/a"
        elem = driver.find_element(By.XPATH, xpath).get_attribute("href")
        addresslist.append(elem)

    return addresslist

# 한 연극 당 최대 1000개까지만 크롤링
max_reviews_per_title = 1000
def scrape_reviews(driver, addresslist):
    total_reviews = []

    for lis in addresslist:
        driver.get(lis)
        time.sleep(1)

        # 팝업창 닫기
        try:
            close_button = driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn")
            close_button.click()
            time.sleep(0.5)
        except NoSuchElementException:
             pass

        title = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[1]/div/div[1]/h2").text

        try:
            nav = driver.find_elements(By.CLASS_NAME, "navItem")
            for i in nav:
                if "관람후기" in i.text:
                    i.click()
                    time.sleep(5)

            reviews_collected = 0
            while reviews_collected < max_reviews_per_title:
                pages = driver.find_element(By.CLASS_NAME, "pageNumWrap").find_elements(By.TAG_NAME, "li")
                for page in pages:
                    page.click()
                    time.sleep(2)
                    for review_item in driver.find_elements(By.CLASS_NAME, "bbsItem"):
                        try:
                            star_element = review_item.find_element(By.CLASS_NAME, "prdStarIcon")
                            star = star_element.get_attribute("data-star")
                        except NoSuchElementException:
                            star = None

                        try:
                            id_element = review_item.find_element(By.CLASS_NAME, "name")
                            id = id_element.text
                        except NoSuchElementException:
                            id = None

                        try:
                            date_element = review_item.find_elements(By.CLASS_NAME, "bbsItemInfoList")[1]
                            date = date_element.text.strip(' "')
                        except (IndexError, NoSuchElementException):
                            date = None

                        if id is not None and star is not None and date is not None:
                            total_reviews.append({'Title': title, 'ID': id, 'Star': star, 'Date': date})
                            reviews_collected += 1

                        if reviews_collected >= max_reviews_per_title:
                            break

                    # print(f"{len(total_reviews)}개의 리뷰를 저장했습니다.")
                    if reviews_collected >= max_reviews_per_title:
                        break

                try:
                    next_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "pageNextBtn.pageArrow")))
                    next_button.click()
                except:
                    break

        except Exception as e:
            print(f"Error while processing {title}: {str(e)}")

    # df = pd.DataFrame(total_reviews, columns=['Title', 'ID', 'Star', 'Date'])

    return total_reviews

driver = webdriver.Chrome()

addresslist = get_theater_links(driver, "http://ticket.interpark.com/TPGoodsList.asp?Ca=Dra&Sort=3")

scraped_data = scrape_reviews(driver, addresslist)
df = pd.DataFrame(scraped_data)

driver.quit()

df.head(10)

In [ ]:
df.tail()

,Title,ID,Star,Date
15491,모든 날 모든 순간,ahsh***,5,2023.09.15
15492,모든 날 모든 순간,waple***,5,2023.09.14
15493,모든 날 모든 순간,jasmine***,5,2023.09.12
15494,모든 날 모든 순간,jerry***,5,2023.09.11
15495,모든 날 모든 순간,cyr***,5,2023.09.11


In [ ]:
df['Title'].unique()

array(['연극 너의 목소리가 들려', '대학로 1위 연극 〈쉬어매드니스〉', '행오버', '연극 한뼘사이',
       '연극〈늘근도둑이야기〉', '뮤직드라마 〈불편한 편의점〉', '코믹감동 휴먼판타지 연극 〈2호선 세입자〉',
       '10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀', '좀비연극 오마이갓', '연극 핫식스',
       '［서울 대학로］연극 라면', '연극 죽여주는 이야기', '연극 〈카페 쥬에네스〉', '연극 〈운빨로맨스〉- 대학로',
       '4D공포연극 〈스위치〉', '연극 〈수상한흥신소〉', '국민 코믹 연극 〈오백에삼십〉 - 대학로 세우아트센터 1관',
       '［2023 SPAF］ 극단 돌파구 〈지상의 여자들〉', '진짜나쁜소녀', '연극 〈뷰티풀라이프〉',
       '★평점9.5★ 코미디의맛/쇼미더퍼니', '이머시브씨어터 카지노', '코미디 연극〈딜리버리〉', '연극 〈굿닥터〉',
       '2시간탈출 졸탄쇼', '연극 〈극적인 하룻밤〉', '연극 〈나의PS파트너〉', '공포연극 ［자취］',
       '!로맨틱코미디 연극〈운빨로맨스〉! - 대구', '대학로 청소년연극 〈사춘기메들리〉',
       '(리얼타임 코믹연극) 택시안에서 - 부산', '(리얼타임 코믹연극) 택시안에서 - 서울',
       '(코믹연극) 달동네-부산', '3대가 웃고 우는 연극 〈염쟁이 유씨〉', '공포스릴러연극〈두여자〉- 대구',
       '공포연극 조각', '그곳에 있었다', '나의 장례식에 와줘', '로맨틱코미디 〈슬기로운 신혼생활〉',
       '모든 날 모든 순간'], dtype=object)

In [ ]:
df.to_excel("C:/Users/82109/Documents/연극_평점.xlsx", sheet_name='Sheet1')

In [ ]:
# 모든 페이지—모든 리뷰 크롤링 (베스트 리뷰 제외하고 크롤링 - 시간 오래 걸림)

In [ ]:
def initialize_driver():
    driver = webdriver.Chrome()
    return driver

def get_theater_links(driver, query):
    driver.get(query)
    time.sleep(1)
    number = 1

    addresslist = []

    for i in range(1, int(number) + 1):
        xpath = f"/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody/tr[{i}]/td[1]/a"
        elem = driver.find_element(By.XPATH, xpath).get_attribute("href")
        addresslist.append(elem)

    return addresslist

# 한 연극 당 최대 리뷰 개수 지정
max_reviews_per_title = 500
def scrape_reviews(driver, addresslist):
    total_reviews = []

    for lis in addresslist:
        driver.get(lis)
        time.sleep(1)

        # 팝업창 닫기
        try:
            close_button = driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn")
            close_button.click()
            time.sleep(0.5)
        except NoSuchElementException:
            pass

        title_element = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[1]/div/div[1]/h2")
        title = title_element.text

        try:
            nav = driver.find_elements(By.CLASS_NAME, "navItem")
            for i in nav:
                if "관람후기" in i.text:
                    i.click()
                    time.sleep(5)

            reviews_collected = 0
            while reviews_collected < max_reviews_per_title:
                pages = driver.find_element(By.CLASS_NAME, "pageNumWrap").find_elements(By.TAG_NAME, "li")
                for page in pages:
                    page.click()
                    time.sleep(2)
                    for review_item in driver.find_elements(By.CLASS_NAME, "bbsItem"):

                        # 각 페이지마다 베스트 리뷰는 제외하고 크롤링
                        try:
                            badge_elements = review_item.find_elements(By.CSS_SELECTOR, ".bastBadge")
                            if not badge_elements:
                                star_element = review_item.find_element(By.CLASS_NAME, "prdStarIcon")
                                star = star_element.get_attribute("data-star")

                                id_element = review_item.find_element(By.CLASS_NAME, "name")
                                id = id_element.text

                                date_element = review_item.find_elements(By.CLASS_NAME, "bbsItemInfoList")[1]
                                date = date_element.text.strip(' "')

                                total_reviews.append({'Title': title, 'ID': id, 'Star': star, 'Date': date})
                                reviews_collected += 1

                            if reviews_collected >= max_reviews_per_title:
                                break

                    if reviews_collected >= max_reviews_per_title:
                        break

                try:
                    next_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, "pageNextBtn.pageArrow")))
                    next_button.click()
                except:
                    break

        except Exception as e:
            print(f"Error while processing {title}: {str(e)}")

    return total_reviews

driver = webdriver.Chrome()

addresslist = get_theater_links(driver, "http://ticket.interpark.com/TPGoodsList.asp?Ca=Dra&Sort=3")

scraped_data = scrape_reviews(driver, addresslist)
df = pd.DataFrame(scraped_data)

driver.quit()

df.head(10)

15개의 리뷰를 저장했습니다.
30개의 리뷰를 저장했습니다.
45개의 리뷰를 저장했습니다.
60개의 리뷰를 저장했습니다.
75개의 리뷰를 저장했습니다.
90개의 리뷰를 저장했습니다.
105개의 리뷰를 저장했습니다.
120개의 리뷰를 저장했습니다.
135개의 리뷰를 저장했습니다.
150개의 리뷰를 저장했습니다.
165개의 리뷰를 저장했습니다.
180개의 리뷰를 저장했습니다.
195개의 리뷰를 저장했습니다.
210개의 리뷰를 저장했습니다.
225개의 리뷰를 저장했습니다.
240개의 리뷰를 저장했습니다.
255개의 리뷰를 저장했습니다.
270개의 리뷰를 저장했습니다.
276개의 리뷰를 저장했습니다.


## 캐스팅 배우

In [ ]:
def initialize_driver():
    driver = webdriver.Chrome()
    return driver

def get_theater_links(driver, query):
    driver.get(query)
    time.sleep(1)
    number = 3

    addresslist = []

    for i in range(1, int(number) + 1):
        xpath = f"/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody/tr[{i}]/td[1]/a"
        elem = driver.find_element(By.XPATH, xpath).get_attribute("href")
        addresslist.append(elem)

    return addresslist

def scrape_reviews(driver, addresslist):
    cast = []

    for lis in addresslist:
        driver.get(lis)
        time.sleep(1)

        # 팝업창 닫기
        try:
            close_button = driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn")
            close_button.click()
            time.sleep(0.5)
        except NoSuchElementException:
             pass

        title = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[1]/div/div[1]/h2").text
        age_percentages = {}

        try:
            nav = driver.find_elements(By.CLASS_NAME, "navItem")
            for i in nav:
                if "판매정보" in i.text:
                    i.click()
                    time.sleep(2)

            cast_element = driver.find_element(By.XPATH, "//th[text()='주연']/following-sibling::td/span[@class='moreInfoCast']")
            cast_info = cast_element.text
            cast.append({'Title': title, 'cast_act' : cast_info})

        except Exception as e:
            print(f"Error while processing {title}: {str(e)}")

    return cast

driver = webdriver.Chrome()

addresslist = get_theater_links(driver, "http://ticket.interpark.com/TPGoodsList.asp?Ca=Dra&Sort=3")

scraped_data = scrape_reviews(driver, addresslist)
df = pd.DataFrame(scraped_data)

driver.quit()

df.head(10)

,Title,cast_act
0,연극 너의 목소리가 들려,"고영빈, 진승욱, 김정원, 피설아, 아인, 박선영, 김하늘, 윤계열, 박건엽, 정성..."
1,대학로 1위 연극 〈쉬어매드니스〉,"임정균, 김태성, 이승복, 석민주, 리재, 노다윗, 백재민, 김채율, 이른봄, 고동..."
2,행오버,"최선량, 정현규, 조영광, 신재명, 손준표, 윤건웅, 최창빈, 김민우, 이도희, 이..."


In [ ]:
df['cast_act']

0    고영빈, 진승욱, 김정원, 피설아, 아인, 박선영, 김하늘, 윤계열, 박건엽, 정성...
1    임정균, 김태성, 이승복, 석민주, 리재, 노다윗, 백재민, 김채율, 이른봄, 고동...
2    최선량, 정현규, 조영광, 신재명, 손준표, 윤건웅, 최창빈, 김민우, 이도희, 이...
Name: cast_act, dtype: object